In [ ]:
# 👥 Telco Customer Analytics & Insights

This notebook provides comprehensive analysis of customer usage patterns, service plans, and revenue optimization opportunities for telecommunications operations.

## Key Areas:
- Customer segmentation and behavior analysis
- Service plan performance and optimization
- Revenue analysis and forecasting
- Customer satisfaction and retention insights


In [ ]:
# Import libraries and establish connection
import snowflake.snowpark as snowpark
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Get Snowpark session and set context
session = snowpark.context.get_active_session()
print(f"❄️ Connected to Snowflake: {session.get_current_account()}")
print(f"🗄️ Current database: {session.get_current_database()}")
print(f"📂 Current schema: {session.get_current_schema()}")

# Check if required tables exist
print("\n🔍 Checking for required tables...")
try:
    tables_check = session.sql("SHOW TABLES LIKE 'CUSTOMER_USAGE'").collect()
    if tables_check:
        print("✅ CUSTOMER_USAGE table found")
    else:
        print("❌ CUSTOMER_USAGE table not found")
        print("💡 Please run configure_attendee_account.template.sql first to create sample data")
except Exception as e:
    print(f"⚠️ Error checking tables: {e}")
    print("💡 Make sure you have the proper database and schema context")


In [ ]:
# Customer Usage Analysis
print("📊 Analyzing customer usage patterns...")

# Get current database and schema for fully qualified table names
current_db = session.get_current_database()
current_schema = session.get_current_schema()

customer_analysis_query = f"""
SELECT 
    service_plan,
    device_type,
    COUNT(DISTINCT customer_id) as customer_count,
    AVG(data_usage_gb) as avg_data_usage,
    AVG(voice_minutes) as avg_voice_minutes,
    AVG(monthly_bill_amount) as avg_monthly_bill
FROM {current_db}.{current_schema}.CUSTOMER_USAGE 
WHERE usage_date >= DATEADD(month, -1, CURRENT_DATE())
GROUP BY service_plan, device_type
ORDER BY customer_count DESC
"""

try:
    customer_df = session.sql(customer_analysis_query).to_pandas()
    print("✅ Customer Usage Summary by Service Plan & Device:")
    display(customer_df)
    
    if customer_df.empty:
        print("📭 No customer usage data found. Please check if data has been loaded.")
        print("💡 Run the configure_attendee_account.template.sql script to load sample data.")
        
except Exception as e:
    print(f"❌ Error executing query: {e}")
    print(f"🔍 Trying to query: {current_db}.{current_schema}.CUSTOMER_USAGE")
    print("💡 Solutions:")
    print("1. Run configure_attendee_account.template.sql to create tables and load data")
    print("2. Verify you have SELECT permissions on the CUSTOMER_USAGE table")
    print("3. Check that you're in the correct database and schema context")


In [ ]:
# Create customer segmentation visualization
print("📈 Creating customer segmentation visualization...")

try:
    if 'customer_df' in locals() and not customer_df.empty:
        fig = px.scatter(customer_df, 
                        x='AVG_DATA_USAGE', 
                        y='AVG_MONTHLY_BILL',
                        size='CUSTOMER_COUNT',
                        color='SERVICE_PLAN',
                        hover_data=['DEVICE_TYPE'],
                        title='📊 Customer Segmentation: Data Usage vs Monthly Bill',
                        labels={'AVG_DATA_USAGE': 'Average Data Usage (GB)',
                               'AVG_MONTHLY_BILL': 'Average Monthly Bill ($)'})

        fig.update_layout(height=600, width=900)
        fig.show()

        print("\n💡 Customer Insights:")
        print("🔵 Larger bubbles = More customers in that segment")
        print("📈 Top segments show high-value customers")
        print("🎯 Use this for targeted marketing and service optimization")
    else:
        print("⚠️ No customer data available for visualization")
        print("🔧 Please ensure the CUSTOMER_USAGE table exists and contains data")
        print("📋 Sample data structure expected:")
        print("   - SERVICE_PLAN (text)")
        print("   - DEVICE_TYPE (text)")
        print("   - CUSTOMER_ID (integer)")
        print("   - DATA_USAGE_GB (decimal)")
        print("   - VOICE_MINUTES (integer)")
        print("   - MONTHLY_BILL_AMOUNT (decimal)")
        print("   - USAGE_DATE (date)")
        
except Exception as e:
    print(f"❌ Error creating visualization: {e}")
    print("💡 Make sure the previous cell executed successfully")
